# 导入相关包

In [3]:
# 导入相关包
import os
import pathlib as pl
import pandas as pd
import numpy as np
import re
from io import StringIO
from datetime import datetime,timedelta
import time
from IPython.core.interactiveshell import InteractiveShell
from tqdm.autonotebook import *
import pdfplumber
from tgrocery import Grocery
tqdm.pandas()
InteractiveShell.ast_node_interactivity = "all"
sys.path.append("..")




# PDF解析原始数据 
## 加载数据并采用pdfplumber抽取PDF中的文字和表格


In [4]:
# 数据准备(train_output文件中格式有点问题，需要提前用excel或者wps打开然后另存为excel文件)
train_outputs = pd.read_excel('../datasets/train_output.xlsx')

# 获取pdf中文字和表格
def extract_pdf_content(pdf_path):
    text_list = []
    table_list = []
    with pdfplumber.open(pdf_path) as pdf:
        for index_page in np.arange(0, len(pdf.pages), 1):
            # 读取多页
            page = pdf.pages[index_page]   # 第n页的信息
            text = page.extract_text()
            text_list.append(text)
            table = page.extract_tables()
            for t in table:
                table_list.append(t)
    return text_list, table_list

def get_dir_file(path):
    '''
    输入文件夹位置，输出整理好的dataframe
    '''
    path_list = os.listdir(path)
    id_list = []
    file_path_list = []
    text_list = []
    table_list = []
    for i in tqdm(path_list):
        if '.PDF' in i:
            file_path = path + i
            id_list.append(int(i.split('.')[0]))
            file_path_list.append(file_path)
            try:
                text_temp, table_temp = extract_pdf_content(file_path)
            except Exception:
                print('此pdf无法读取')
                text_temp, table_temp = [], []
            text_list.append(text_temp)
            table_list.append(table_temp)
            
    df = pd.DataFrame()
    df['sample_id'] = id_list
    df['file_path'] = file_path_list
    df['text'] = text_list
    df['tabel'] = table_list
    df = df.sort_values('sample_id')
    return df

# 文件处理太慢，可持续化保存文件
train_path = '../datasets/train.csv'
if os.path.exists(train_path):
    train_df = pd.read_csv(train_path)
else:
    train_df = get_dir_file('datasets/train_data/')
    train_df.to_csv(train_path,index=False)
    train_df = pd.read_csv(train_path)

test_path =  '../datasets/test.csv'
if os.path.exists(test_path):
    test_df = pd.read_csv(test_path)
else:
    test_df = get_dir_file('datasets/test_data/')
    test_df.to_csv(test_path,index=False)
    test_df = pd.read_csv(test_path)

train_outputs.head(2)
train_df.head(2)
test_df.head(2)

,sample_id,认购日期,理财产品名称,产品发行方名称,理财类型,认购金额(万元),产品起息日,产品到息日,产品期限,资金来源,实际购买公司名称,实际购买公司和上市公司关系,买卖方是否有关联关系,公告日期
0,1,2019-03-27,汇聚金1号,中融国际信托有限公司,信托,10000.0,2019-03-27,2019-09-23,180天,自有资金,恒生电子股份有限公司,公司本身,否,2019-04-25
1,1,2019-03-27,招商银行步步生金8699,招商银行,银行理财产品,200.0,2019-03-27,NaT,NaN,自有资金,恒生电子股份有限公司,公司本身,否,2019-04-25


,sample_id,file_path,text,tabel
0,1,datasets/train_data/1.PDF,[' ...,"[[['', None, None, '', None, None, '', None, N..."
1,2,datasets/train_data/2.PDF,[' ...,"[[['', None, None, '', None, None, '', None, N..."


,sample_id,file_path,text,tabel
0,11188,datasets/test_data/11188.PDF,['北京京西文化旅游股份有限公司监事会\n \n \n关于使用部分闲置募集资金购买理财产品的...,[]
1,11189,datasets/test_data/11189.PDF,['北京京西文化旅游股份有限公司 \n监事会关于使用部分自有资金购买理财产品的意见 \n根据...,[]


In [5]:
# 构造训练集验证集
train_df = train_df.sample(frac=1, random_state=1017)
val_df = train_df[:1800]
train_df = train_df[1800:]

训练字段分类器

In [6]:
# 最后一部分字段采用预测好的部分，跟提取的text做交互采用双输入lstm在dense层做交互预测最后几个字段

# train_lstm_input = pd.merge(train_df, train_outputs, on='sample_id', how='left')
# result_matrix
train_lstm_input = pd.merge(train_df, train_outputs, on='sample_id', how='left')

train_lstm_input = train_lstm_input.fillna('否')

# label_1理财类型-10  label_2资金来源-3 label_3实际购买公司和上市公司关系-3 label_4买卖方是否有关联关系-2
from sklearn.preprocessing import LabelEncoder
label_1 = LabelEncoder()
# label_2 = LabelEncoder()
# label_3 = LabelEncoder()
# label_4 = LabelEncoder()

train_data = pd.DataFrame()
tmp=pd.DataFrame()
train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) 

# train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) + '_' + train_lstm_input['产品发行方名称'].astype(str)

# train_data['text_2'] = train_lstm_input['text'].astype(str)

# train_lstm_input["文本类别"]="理财产品"

train_data['label_1'] = "理财产品"


train_data2=train_lstm_input[train_lstm_input["产品发行方名称"]!="否"].reset_index(drop=True)

# train_data2["文本类别"]="发行方"

tmp['text_1']=train_data2["产品发行方名称"].astype(str)

# tmp['text_2']= train_data2["text"].astype(str)

tmp['label_1']="发行方"

train_data = pd.concat([train_data,tmp]).reset_index(drop=True)

train_data2=train_lstm_input[train_lstm_input["实际购买公司名称"]!="否"].reset_index(drop=True)

# train_data2["文本类别"]="发行方"

tmp['text_1']=train_data2["实际购买公司名称"].astype(str)

# tmp['text_2']= train_data2["text"].astype(str)

tmp['label_1']="购买公司"

train_data = pd.concat([train_data,tmp]).reset_index(drop=True)


other_columns_list=["认购金额(万元)","认购日期","资金来源","实际购买公司和上市公司关系"]

for item in other_columns_list:

    train_data2=train_lstm_input[train_lstm_input[item]!="否"].reset_index(drop=True)

    # train_data2["文本类别"]=item

    tmp['text_1']=train_data2[item].astype(str)

    # tmp['text_2']= train_data2["text"].astype(str)

    tmp['label_1']="其它"

    
    train_data = pd.concat([train_data,tmp]).reset_index(drop=True)



# train_data['label_2'] = label_2.fit_transform(train_lstm_input['资金来源'])
# train_data['label_3'] = label_3.fit_transform(train_lstm_input['实际购买公司和上市公司关系'])
# train_data['label_4'] = label_4.fit_transform(train_lstm_input['买卖方是否有关联关系'])
train_data

train_src=[]
for text,label in train_data[["text_1","label_1"]].values:
    train_src.append([label,text])


grocery=Grocery("productOrcounter")


grocery.train(train_src)

grocery.save()



,text_1,label_1
0,中银保本理财-人民币按期开放理财产品,理财产品
1,中银保本理财-人民币按期开放理财产品,理财产品
2,与利率挂钩的结构性产品,理财产品
3,广发银行“薪加薪”16号XJXCKJ2578,理财产品
4,兴业银行“金雪球-优悦”保本开放式人民币理财产品(2M),理财产品
...,...,...
185903,控股参股公司,其它
185904,控股参股公司,其它
185905,控股参股公司,其它
185906,控股参股公司,其它


Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\hp\AppData\Local\Temp\jieba.cache
Loading model cost 1.118 seconds.
Prefix dict has been built successfully.


全文公司字段的抽取

In [214]:
# 最后一部分字段采用预测好的部分，跟提取的text做交互采用双输入lstm在dense层做交互预测最后几个字段

# train_lstm_input = pd.merge(train_df, train_outputs, on='sample_id', how='left')
# result_matrix
train_lstm_input = pd.merge(train_df, train_outputs, on='sample_id', how='left')

train_lstm_input = train_lstm_input.fillna('否')

# label_1理财类型-10  label_2资金来源-3 label_3实际购买公司和上市公司关系-3 label_4买卖方是否有关联关系-2
from sklearn.preprocessing import LabelEncoder
label_1 = LabelEncoder()
# label_2 = LabelEncoder()
# label_3 = LabelEncoder()
# label_4 = LabelEncoder()

train_data = pd.DataFrame()
tmp=pd.DataFrame()
train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) 

# train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) + '_' + train_lstm_input['产品发行方名称'].astype(str)

# train_data['text_2'] = train_lstm_input['text'].astype(str)

# train_lstm_input["文本类别"]="理财产品"

train_data['label_1'] = "理财产品"


train_data2=train_lstm_input[train_lstm_input["产品发行方名称"]!="否"].reset_index(drop=True)

# train_data2["文本类别"]="发行方"

tmp['text_1']=train_data2["产品发行方名称"].astype(str)

# tmp['text_2']= train_data2["text"].astype(str)

tmp['label_1']="发行方"

train_data = pd.concat([train_data,tmp]).reset_index(drop=True)

train_data2=train_lstm_input[train_lstm_input["实际购买公司名称"]!="否"].reset_index(drop=True)

# train_data2["文本类别"]="发行方"

tmp['text_1']=train_data2["实际购买公司名称"].astype(str)

# tmp['text_2']= train_data2["text"].astype(str)

tmp['label_1']="购买公司"

train_data = pd.concat([train_data,tmp]).reset_index(drop=True)


other_columns_list=["认购金额(万元)","认购日期","资金来源","实际购买公司和上市公司关系"]

for item in other_columns_list:

    train_data2=train_lstm_input[train_lstm_input[item]!="否"].reset_index(drop=True)

    # train_data2["文本类别"]=item

    tmp['text_1']=train_data2[item].astype(str)

    # tmp['text_2']= train_data2["text"].astype(str)

    tmp['label_1']="其它"

    
    train_data = pd.concat([train_data,tmp]).reset_index(drop=True)



# train_data['label_2'] = label_2.fit_transform(train_lstm_input['资金来源'])
# train_data['label_3'] = label_3.fit_transform(train_lstm_input['实际购买公司和上市公司关系'])
# train_data['label_4'] = label_4.fit_transform(train_lstm_input['买卖方是否有关联关系'])
train_data

train_src=[]
for text,label in train_data[["text_1","label_1"]].values:
    train_src.append([label,text])


grocery_word_selector=Grocery("wordSelector")


grocery_word_selector.train(train_src)

grocery_word_selector.save()



,text_1,label_1
0,中银保本理财-人民币按期开放理财产品,理财产品
1,中银保本理财-人民币按期开放理财产品,理财产品
2,与利率挂钩的结构性产品,理财产品
3,广发银行“薪加薪”16号XJXCKJ2578,理财产品
4,兴业银行“金雪球-优悦”保本开放式人民币理财产品(2M),理财产品
...,...,...
185903,控股参股公司,其它
185904,控股参股公司,其它
185905,控股参股公司,其它
185906,控股参股公司,其它


# 数据处理
## 抽取整体数据（一个sampleid内此字段内容都相同）
## 公告时间，实际购买公司

#### 1.抽取公告时间

In [5]:
# 首先针对任务抽取时间（每个时间跟每个id是一一对应的）
# 要不是取第一个时间，要不就是取最后一个时间（或者时间加一）这里可以建立一个模型预测
# base这里面直接取最后一个时间作为发布日期

CN_NUM = {
    u'〇': 0, u'一': 1, u'二': 2, u'三': 3,
    u'四': 4, u'五': 5, u'六': 6, u'七': 7,
    u'八': 8, u'九': 9, u'零': 0, u'壹': 1,
    u'贰': 2, u'叁': 3, u'肆': 4, u'伍': 5,
    u'陆': 6, u'柒': 7, u'捌': 8, u'玖': 9,
    u'貮': 2, u'两': 2,
}


def get_put_time_from_text(row):
    row = row.replace(' ', '').replace('\\n', '')
    for key in CN_NUM:
        row = row.replace(key, str(CN_NUM[key]))   
    r = row.replace("年", "-").replace("月", "-").replace("日", " ").replace("/", "-").strip()
    regex = "(\d{4}-\d{1,2}-\d{1,2})"
    r = re.findall(regex, r)
    if len(r)==0:
        return np.nan
    time_str = r[-1]
    first = time_str.split('-')[0]
    second = time_str.split('-')[1]
    last = time_str.split('-')[-1]
    second = str.zfill(second, 2)
    last = str.zfill(last, 2)
    r = '-'.join([first, second, last])
    return r

val_result = pd.DataFrame()
val_result['sample_id'] = val_df['sample_id']
val_result['predict_time'] = val_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)
test_gg = train_outputs.groupby('sample_id').apply(lambda row:list(row['公告日期'])[0]).reset_index()
test_gg.columns = ['sample_id', 'time']
val_result = pd.merge(val_result, test_gg, on='sample_id', how='left')

# 判断验证集的准确率
np.sum(val_result['predict_time'].astype(str) == val_result['time'].astype(str))/len(val_result)

val_time = val_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)
# test_time = test_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)

100%|██████████| 1800/1800 [00:00<00:00, 3419.06it/s]


0.4583333333333333

100%|██████████| 1800/1800 [00:00<00:00, 4030.98it/s]


#### 2.抽取实际购买公司

In [6]:
# 抽取购买公司
# 前几句话出现
# 将其按照\\n 和空格切割
def get_gm(row):
    result = re.split('[\\\\n ]',row)
    for i in result:
        if '公司' in i:
            return i

val_gm = val_df.progress_apply(lambda row:get_gm(row['text']), axis=1)
# test_gm = test_df.progress_apply(lambda row:get_gm(row['text']), axis=1)

100%|██████████| 1800/1800 [00:00<00:00, 3258.32it/s]


#### 3.清洗提取出来的tabel数据，主要是清洗掉有问题的列 
# 重写清洗方法

In [7]:
# tmp_table=pd.concat([val_df["sample_id"],pd.DataFrame(val_df_tabel)],axis=1)
# tmp_table["len_table"]=tmp_table["tabel"].apply(lambda x:[len(i) for i in x])
# # tmp_table.to_excel("table矩阵检验1.xlsx")


a=[1,2,3,4,5,6]
b=[0,2,3,4]
a=pd.DataFrame(a)
a.loc[b]="aaaa"
a

,0
0,aaaa
1,2
2,aaaa
3,aaaa
4,aaaa
5,6


#大量PDF的理财购买信息不在table中，而是在text中，此段为判断理财信息是否只需要从text中提取

In [8]:
# tabel_tmp_value=eval(val_df[val_df["sample_id"]==35]["tabel"].iloc[0])
# # tabel_tmp_value_list=[]
# table_value_df=pd.DataFrame(tabel_tmp_value)
val_df.loc[1419].shape[0]
# tmp2=val_df.head(51).index
# list(tmp1)
# np.array_equal(tmp1,tmp2)
# set(tmp2).issubset(tmp1)
# set(tmp2).difference(tmp1)

4

In [196]:
# tabel_tmp_value=eval(val_df[val_df["sample_id"]==4333]["tabel"].iloc[0])
# tabel_tmp_value=eval(val_df[val_df["sample_id"]==35]["tabel"].iloc[0])
# tabel_tmp_value_list=[]
# table_value_df=pd.DataFrame(tabel_tmp_value[0])
# table_value_df.head(20)
# for item in tabel_tmp_value:

column_name_judge_word=["民币）","民币)","万元)","元)","万元）","元）","%)","%）","金额"]


#寻找有有效值存在的列
def find_valid_columns(df):
    count_list=[]
    result=[]
    df.copy().T.apply(lambda x:count_list.append(get_valid_columns_num(x.T)),axis=1)
    # df.copy().T.apply(lambda x:print(x),axis=1)
    # print(count_list)
    # print(count_list)
    for i in range(len(count_list)):
        if count_list[i]>(int(df.shape[0]/7)):
            result.append(i)
    # print(result)   
    return result


#寻找（字段）所在的理论最后一行
def find_amt(df):
    global column_name_judge_word
    max_index=-1
    df=df.head(7)
    for index in range(df.shape[0]):
        for item in column_name_judge_word:
            flag=[]
            flag_num=[]
            df.loc[index].map(lambda x:flag_num.append(re.match("[ ]*?\d",x)))
            df.loc[index].map(lambda x:flag.append((item in x)))
            for i in flag_num:
                if(i is not None):
                    return max_index
            if True in flag:
                max_index=index
    return max_index

column_name_judge_word=["民币）","民币)","万元)","元)","万元）","元）","%)","%）","金额"]


#寻找单个理财产品所在的理论最后一行
def find_product(df):
    global column_name_judge_word
    max_index=-1
    df=df.head(7)
    for index in range(df.shape[0]):
        for item in column_name_judge_word:
            flag=[]
            flag_num=[]
            df.loc[index].map(lambda x:flag_num.append(re.match("[ ]*?\d",x)))
            df.loc[index].map(lambda x:flag.append((item in x)))
            for i in flag_num:
                if(i is not None):
                    return max_index
            if True in flag:
                max_index=index
    return max_index

def get_valid_columns_num(df):
    df=df.T.dropna(axis=0).apply(lambda x:x.replace(" ","")).to_frame().T.reset_index(drop=True).T
    # print(df)
    return df[(df[0]!="")&(df[0]!=np.nan)].shape[0]

def get_valid_columns_index(df):
    df=df.T.dropna(axis=0).apply(lambda x:x.replace(" ","")).to_frame().T.reset_index(drop=True).T
    # print(df)
    return df[(df[0]!="")&(df[0]!=np.nan)].index

def field_location_optimization(columns_list,df):
    if(df.shape[1]!=columns_list.shape[0]):
        return columns_list.to_frame().T.head(0)
    max_columns_num=get_valid_columns_num(columns_list)
    if(df.shape[0]==0):
        return df
    valid_clomuns_index=list(get_valid_columns_index(columns_list))
    # print(valid_clomuns_index)
    result_df=df.head(0)
    # print(result_df.shape)
    # print(df)
    for index in range(df.shape[0]):
        tmp_row=df.loc[index]
        tmp_index=get_valid_columns_index(tmp_row)
        # print(list(tmp_index))
        if set(tmp_index).issubset(valid_clomuns_index):
            pass
        else:
            difference_list=list(set(tmp_index).difference(valid_clomuns_index))
            
            location_list=[]
            tmp_difference_list=difference_list.copy()
            for item in difference_list:
                location=item
                #前后浮动位移（可优化）
                for i in range(1,3):
                    location=(item-i)
                    if location in valid_clomuns_index and location not in tmp_difference_list:
                        location_list.append(location)
                        tmp_difference_list.remove(item)
                        tmp_difference_list.append(location)
                        break
                    location=(item+i)
                    if location in valid_clomuns_index and location not in tmp_difference_list:
                        location_list.append(location)
                        tmp_difference_list.remove(item)
                        tmp_difference_list.append(location)
                        break
            result_location=list(df.columns)
            # print(location_list)
            # print(difference_list)
            # print("---------------------")
            for i in range(len(location_list)):
                result_location[difference_list[i]]=location_list[i]
                result_location[location_list[i]]=difference_list[i]
            # print(tmp_row)
            tmp_row=tmp_row.iloc[result_location].reset_index(drop=True)
            # print(tmp_row)
            # print(result_location)
        result_df=tmp_row.to_frame().T if result_df is None else pd.concat([result_df,tmp_row.to_frame().T])
    # print(np.array_equal(get_valid_columns_index(columns_list),get_valid_columns_index(result_df)))
    # print(list(set(get_valid_columns_index(columns_list)).difference(result_df)))
    # print(result_df)
    # print(result_df.shape)
    return result_df
def row_combine(sample_id,pdf_table):
# 将字符串转化成多维列表
    pdf_table=eval(pdf_table)

    table_result=[]
    start_rows_list=[]
    first_line_list=[]
    product_df_list=[]
    for item in pdf_table:
        #考虑加入一个判定表格位置的方法（暂缺）

        tmp_table_df=pd.DataFrame(item)
        tmp_table_df=tmp_table_df.fillna("").applymap(lambda x: x.replace("\n",""))
        #清除全空行
        drop_index_list=[]
        for index in range(tmp_table_df.shape[0]):
            judge=[]
            noshow=tmp_table_df.loc[index].map(lambda x:judge.append(x==""))
            if(False not in judge):
                drop_index_list.append(index)
        
        tmp_table_df=tmp_table_df.drop(drop_index_list).reset_index(drop=True)

        base_row=find_amt(tmp_table_df)

        tmp_row=None

        len_product_df_list=len(product_df_list)
        if base_row!=-1 and len_product_df_list!=0 :
            # print(product_df_list[len_product_df_list-1].shape)
            # print(table_result[len_product_df_list-1])
            if (product_df_list[len_product_df_list-1] is not None ) and (product_df_list[len_product_df_list-1].shape[0]==0):
                # print(table_result[len_product_df_list-1])
                tmp_row=table_result[len_product_df_list-1]

        if base_row==-1:
            if(len(table_result)>0):
                product_df=field_location_optimization(table_result[len(table_result)-1],tmp_table_df.loc[:].reset_index(drop=True))
                if(product_df is not None):
                    len_product_df_list=len(product_df_list)-1
                    product_df_list[len_product_df_list]=pd.concat([product_df_list[len_product_df_list],product_df])
                    # print(product_df.shape)
                else:
                    pass
            continue

        valid_columns_nums=[]
        len_df=None
        for index in range(0,base_row+1):
            len_list=tmp_table_df.loc[index].map(lambda x:len(x))
            for i in range(len(len_list)):
                if len_list[i]==0 and len_df is not None:
                    len_list[i]=len_df.tail(1)[i]
            len_df=pd.DataFrame(len_list).T if len_df is None else pd.concat([len_df,pd.DataFrame(len_list).T]).reset_index(drop=True)
            row_result=tmp_table_df.loc[index].map(lambda x:re.sub(r"$[ ]+?","",re.sub(r"^[ ]+?","",x)))
            tmp_row= row_result if tmp_row is None else tmp_row+row_result
            # valid_columns_nums.append(get_valid_columns_num(tmp_row))
        
        start_row=base_row+1

        for index in range(base_row+1,tmp_table_df.shape[0]):
            len_list=tmp_table_df.loc[index].map(lambda x:len(x))
            len_judge=(pd.DataFrame(len_list).T-len_df.loc[len_df.shape[0]-1]).reset_index(drop=True)
            len_judge=len_judge.T
            if(len_judge[len_judge[0]>0].shape[0]>0):
                break
            
            start_row+=1
            for i in range(len(len_list)):
                if len_list[i]==0 and len_df is not None:
                    len_list[i]=len_df.tail(1)[i]
            len_df=pd.DataFrame(len_list).T if len_df is None else pd.concat([len_df,pd.DataFrame(len_list).T]).reset_index(drop=True)
            row_result=tmp_table_df.loc[index].map(lambda x:re.sub(r"$[ ]+?","",re.sub(r"^[ ]+?","",x)))
            tmp_row= row_result if tmp_row is None else tmp_row+row_result

            # valid_columns_nums.append(get_valid_columns_num(tmp_row))
        
        columns_list=tmp_row.dropna().map(lambda x:x.replace(" ",""))
        valid_columns=find_valid_columns(tmp_table_df.iloc[start_row:,:])
        tmp_valid_columns=get_valid_columns_index(columns_list)
        # print(valid_columns)
        # print(tmp_valid_columns)

        difference=list(set(valid_columns).difference(tmp_valid_columns))
        # print(difference)


        # print("-------------")
        if(len(difference)>2):
            # print("---------")
            union_set=set(valid_columns).union(tmp_valid_columns)
            tmp_row[union_set]="添加字段"
            columns_list=tmp_row.dropna().map(lambda x:x.replace(" ",""))


        
        #处理错列问题
        product_df=field_location_optimization(columns_list,tmp_table_df.loc[start_row:].reset_index(drop=True))
        
        first_line=[]
        tmp_table_df.head(2).applymap(lambda x:first_line.append(x))
        first_line_result=[]
        for i in first_line:
            i=re.sub(r"^[ ]*?","",i)
            i=re.sub(r"$[ ]*?","",i)
            # i=re.sub(r"[ ]*?","",i)
            if i != "":
                first_line_result.append(i)
        table_result.append(columns_list)
        first_line_list.append(first_line_result)
        start_rows_list.append(start_row)
        product_df_list.append(product_df)
        # print(len(table_result))
        # print(len(first_line_list))
        # print(len(start_rows_list))
    return table_result,start_rows_list,first_line_list,product_df_list


tmp_table_column={}
tmp_table_column["sample_id"]=[]
tmp_table_column["columns"]=[]
tmp_table_column["start_row"]=[]
tmp_table_column["first_line"]=[]
tmp_table_column["product_df"]=[]
for sample_id,tabel in tqdm(val_df[["sample_id","tabel"]].values):##修改使用的数据集
    table_result,start_rows_list,first_line_list,product_df_list=row_combine(sample_id,tabel)
    for i in range(len(start_rows_list)):
        valid_columns_index=get_valid_columns_index(table_result[i])
        tmp_table_column["sample_id"].append(sample_id)
        tmp_table_column["columns"].append(table_result[i].loc[valid_columns_index].T.reset_index(drop=True).T)
        tmp_table_column["start_row"].append(start_rows_list[i])
        tmp_table_column["first_line"].append(first_line_list[i])
        tmp_table_column["product_df"].append(product_df_list[i][valid_columns_index].T.reset_index(drop=True).T)

tmp_table_column=pd.DataFrame(tmp_table_column)


# tmp_table_column[["columns","start_row","first_line"]]=val_df["tabel"].head(5).apply(lambda x:row_combine(x))


# tmp=val_df.iloc[177,:][["sample_id","tabel"]]
# table_result,start_rows_list,first_line_list,product_df_list=row_combine(tmp["sample_id"],tmp["tabel"])
# table_result
# product_df_list

100%|██████████| 1800/1800 [07:40<00:00,  3.91it/s]


# tabel_tmp_value=eval(val_df[val_df["sample_id"]==4333]["tabel"].iloc[0])
# tabel_tmp_value=eval(val_df[val_df["sample_id"]==35]["tabel"].iloc[0])
# tabel_tmp_value_list=[]
# table_value_df=pd.DataFrame(tabel_tmp_value[0])
# table_value_df.head(20)
# for item in tabel_tmp_value:

column_name_judge_word=["民币）","民币)","万元)","元)","万元）","元）","%)","%）","金额"]


#寻找有有效值存在的列
def find_valid_columns(df):
    count_list=[]
    result=[]
    df.copy().T.apply(lambda x:count_list.append(get_valid_columns_num(x.T)),axis=1)
    # df.copy().T.apply(lambda x:print(x),axis=1)
    # print(count_list)
    # print(count_list)
    for i in range(len(count_list)):
        if count_list[i]>(int(df.shape[0]/7)):
            result.append(i)
    # print(result)   
    return result


#寻找（字段）所在的理论最后一行
def find_amt(df):
    global column_name_judge_word
    max_index=-1
    df=df.head(7)
    for index in range(df.shape[0]):
        for item in column_name_judge_word:
            flag=[]
            flag_num=[]
            df.loc[index].map(lambda x:flag_num.append(re.match("[ ]*?\d",x)))
            df.loc[index].map(lambda x:flag.append((item in x)))
            for i in flag_num:
                if(i is not None):
                    return max_index
            if True in flag:
                max_index=index
    return max_index

column_name_judge_word=["民币）","民币)","万元)","元)","万元）","元）","%)","%）","金额"]


#寻找单个理财产品所在的理论最后一行
def find_product(df):
    global column_name_judge_word
    max_index=-1
    df=df.head(7)
    for index in range(df.shape[0]):
        for item in column_name_judge_word:
            flag=[]
            flag_num=[]
            df.loc[index].map(lambda x:flag_num.append(re.match("[ ]*?\d",x)))
            df.loc[index].map(lambda x:flag.append((item in x)))
            for i in flag_num:
                if(i is not None):
                    return max_index
            if True in flag:
                max_index=index
    return max_index

def get_valid_columns_num(df):
    df=df.T.dropna(axis=0).apply(lambda x:x.replace(" ","")).to_frame().T.reset_index(drop=True).T
    # print(df)
    return df[(df[0]!="")&(df[0]!=np.nan)].shape[0]

def get_valid_columns_index(df):
    df=df.T.dropna(axis=0).apply(lambda x:x.replace(" ","")).to_frame().T.reset_index(drop=True).T
    # print(df)
    return df[(df[0]!="")&(df[0]!=np.nan)].index

def field_location_optimization(columns_list,df):
    if(df.shape[1]!=columns_list.shape[0]):
        return columns_list.to_frame().T.head(0)
    max_columns_num=get_valid_columns_num(columns_list)
    if(df.shape[0]==0):
        return df
    valid_clomuns_index=list(get_valid_columns_index(columns_list))
    # print(valid_clomuns_index)
    result_df=df.head(0)
    # print(result_df.shape)
    # print(df)
    for index in range(df.shape[0]):
        tmp_row=df.loc[index]
        tmp_index=get_valid_columns_index(tmp_row)
        # print(list(tmp_index))
        if set(tmp_index).issubset(valid_clomuns_index):
            pass
        else:
            difference_list=list(set(tmp_index).difference(valid_clomuns_index))
            
            location_list=[]
            tmp_difference_list=difference_list.copy()
            for item in difference_list:
                location=item
                #前后浮动位移（可优化）
                for i in range(1,3):
                    location=(item-i)
                    if location in valid_clomuns_index and location not in tmp_difference_list:
                        location_list.append(location)
                        tmp_difference_list.remove(item)
                        tmp_difference_list.append(location)
                        break
                    location=(item+i)
                    if location in valid_clomuns_index and location not in tmp_difference_list:
                        location_list.append(location)
                        tmp_difference_list.remove(item)
                        tmp_difference_list.append(location)
                        break
            result_location=list(df.columns)
            # print(location_list)
            # print(difference_list)
            # print("---------------------")
            for i in range(len(location_list)):
                result_location[difference_list[i]]=location_list[i]
                result_location[location_list[i]]=difference_list[i]
            # print(tmp_row)
            tmp_row=tmp_row.iloc[result_location].reset_index(drop=True)
            # print(tmp_row)
            # print(result_location)
        result_df=tmp_row.to_frame().T if result_df is None else pd.concat([result_df,tmp_row.to_frame().T])
    # print(np.array_equal(get_valid_columns_index(columns_list),get_valid_columns_index(result_df)))
    # print(list(set(get_valid_columns_index(columns_list)).difference(result_df)))
    # print(result_df)
    # print(result_df.shape)
    return result_df
def row_combine(sample_id,pdf_table):
# 将字符串转化成多维列表
    pdf_table=eval(pdf_table)

    table_result=[]
    start_rows_list=[]
    first_line_list=[]
    product_df_list=[]
    tmp_table_result=[]
    tmp_product_df_list=[]
    for item in pdf_table:
        #考虑加入一个判定表格位置的方法（暂缺）

        tmp_table_df=pd.DataFrame(item)
        tmp_table_df=tmp_table_df.fillna("").applymap(lambda x: x.replace("\n",""))
        #清除全空行
        drop_index_list=[]
        for index in range(tmp_table_df.shape[0]):
            judge=[]
            noshow=tmp_table_df.loc[index].map(lambda x:judge.append(x==""))
            if(False not in judge):
                drop_index_list.append(index)
        
        tmp_table_df=tmp_table_df.drop(drop_index_list).reset_index(drop=True)

        base_row=find_amt(tmp_table_df)

        tmp_row=None

        # print(tmp_table_df)

        len_product_df_list=len(product_df_list)
        if base_row!=-1 and len_product_df_list!=0 :
            # print(product_df_list[len_product_df_list-1].shape)
            # print(table_result[len_product_df_list-1])
            if (product_df_list[len_product_df_list-1] is not None ) and (product_df_list[len_product_df_list-1].shape[0]==0):
                # print(table_result[len_product_df_list-1])
                tmp_row=table_result[len_product_df_list-1]

        if base_row==-1:
            if(len(table_result)>0):
                a=0
                # print(tmp_table_df.index)
                for index in tmp_table_df.index:
                    if(get_valid_columns_num(tmp_table_df.loc[index])==1):
                        a=index+1
                    else:
                        break
                for i in table_result:
                    product_df=field_location_optimization(i,tmp_table_df.loc[a:].reset_index(drop=True))
                    if(product_df is not None):
                        # print(i)
                        # print(product_df)
                        tmp_table_result.append(i)
                        tmp_product_df_list.append(product_df)
                        break
                    # print(product_df.shape)
                else:
                    pass
            continue

        valid_columns_nums=[]
        len_df=None
        for index in range(0,base_row+1):
            len_list=tmp_table_df.loc[index].map(lambda x:len(x))
            for i in range(len(len_list)):
                if len_list[i]==0 and len_df is not None:
                    len_list[i]=len_df.tail(1)[i]
            len_df=pd.DataFrame(len_list).T if len_df is None else pd.concat([len_df,pd.DataFrame(len_list).T]).reset_index(drop=True)
            row_result=tmp_table_df.loc[index].map(lambda x:re.sub(r"$[ ]+?","",re.sub(r"^[ ]+?","",x)))
            tmp_row= row_result if tmp_row is None else tmp_row+row_result
            # valid_columns_nums.append(get_valid_columns_num(tmp_row))
        
        start_row=base_row+1

        for index in range(base_row+1,tmp_table_df.shape[0]):
            len_list=tmp_table_df.loc[index].map(lambda x:len(x))
            len_judge=(pd.DataFrame(len_list).T-len_df.loc[len_df.shape[0]-1]).reset_index(drop=True)
            len_judge=len_judge.T
            if(len_judge[len_judge[0]>0].shape[0]>0):
                break
            
            start_row+=1
            for i in range(len(len_list)):
                if len_list[i]==0 and len_df is not None:
                    len_list[i]=len_df.tail(1)[i]
            len_df=pd.DataFrame(len_list).T if len_df is None else pd.concat([len_df,pd.DataFrame(len_list).T]).reset_index(drop=True)
            row_result=tmp_table_df.loc[index].map(lambda x:re.sub(r"$[ ]+?","",re.sub(r"^[ ]+?","",x)))
            tmp_row= row_result if tmp_row is None else tmp_row+row_result

            # valid_columns_nums.append(get_valid_columns_num(tmp_row))
        
        columns_list=tmp_row.dropna().map(lambda x:x.replace(" ",""))
        valid_columns=find_valid_columns(tmp_table_df.iloc[start_row:,:])
        tmp_valid_columns=get_valid_columns_index(columns_list)
        # print(valid_columns)
        # print(tmp_valid_columns)

        difference=list(set(valid_columns)^set(tmp_valid_columns))
        # print(difference)


        # print("-------------")
        if(len(difference)>=1):
            # print("---------")
            union_set=set(valid_columns).difference(tmp_valid_columns)
            tmp_row[union_set]="添加字段"
            columns_list=tmp_row.dropna().map(lambda x:x.replace(" ",""))


        
        #处理错列问题
        product_df=field_location_optimization(columns_list,tmp_table_df.loc[start_row:].reset_index(drop=True))
        
        first_line=[]
        tmp_table_df.head(2).applymap(lambda x:first_line.append(x))
        first_line_result=[]
        for i in first_line:
            i=re.sub(r"^[ ]*?","",i)
            i=re.sub(r"$[ ]*?","",i)
            # i=re.sub(r"[ ]*?","",i)
            if i != "":
                first_line_result.append(i)
        table_result.append(columns_list)
        first_line_list.append(first_line_result)
        start_rows_list.append(start_row)
        product_df_list.append(product_df)
        # print(len(table_result))
        # print(len(first_line_list))
# print(len(start_rows_list))
    for i in tmp_table_result:
        start_rows_list.append(0)
        first_line_list.append("")
    table_result.extend(tmp_table_result)
    # print(tmp_table_result)
    product_df_list.extend(tmp_product_df_list)
    return table_result,start_rows_list,first_line_list,product_df_list


tmp_table_column={}
tmp_table_column["sample_id"]=[]
tmp_table_column["columns"]=[]
tmp_table_column["start_row"]=[]
tmp_table_column["first_line"]=[]
tmp_table_column["product_df"]=[]
for sample_id,tabel in tqdm(val_df[val_df["sample_id"]!=-1][["sample_id","tabel"]].values):##修改使用的数据集
    table_result,start_rows_list,first_line_list,product_df_list=row_combine(sample_id,tabel)
    # print(table_result)
    for i in range(len(start_rows_list)):
        valid_columns_index=get_valid_columns_index(table_result[i])
        tmp_table_column["sample_id"].append(sample_id)
        tmp_table_column["columns"].append(table_result[i].loc[valid_columns_index].T.reset_index(drop=True).T)
        tmp_table_column["start_row"].append(start_rows_list[i])
        tmp_table_column["first_line"].append(first_line_list[i])
        tmp_table_column["product_df"].append(product_df_list[i][valid_columns_index].T.reset_index(drop=True).T)

tmp_table_column=pd.DataFrame(tmp_table_column)



product_df_list[i]
len(valid_columns_index)
product_df_list[i][valid_columns_index]

In [10]:
title_num_char=["一、","二、","三、","四、","五、","六、","七、","八、","九、","十、","十一、","十二、","十三、","十四、","十五、"]
s_title_num_char=["（一）","（二）","（三）","（四）","（五）","（六）","（七）","（八）","（九）","（十）","（十一）","（十二）","（十三）","（十四）","（十五）"]
s_title_num_char.extend(["[(]一[)]","[(]二[)]","[(]三[)]","[(]四[)]","[(]五[)]","[(]六[)]","[(]七[)]","[(]八[)]","[(]九[)]","[(]十[)]","[(]十一[)]","[(]十二[)]","[(]十三[)]","[(]十四[)]","[(]十五[)]"])

title_pos_words=[]
title_neg_words=["备查","日前","过去","履行","审批","程序","风险","措施","影响","累计","赎回","到期","截至","意见","十二个月内","公告前","报备文件","前期"]


def get_title(text):
    global title_num_char
    title_list=[]
    title_type_list=[]
    text_start_iter_list=[]
    text_end_iter_list=[]
    for item in title_num_char:
        pattern = re.compile(item+r"[ ]*?[^ ]+?[ ]")
        tmp=pattern.finditer(text)
        for i in tmp:
            title_list.append(i.group())
            text_start_iter_list.append(i.span(0)[0])
            title_type_list.append(1)
            text_end_iter_list.append(i.span(0)[1])
    
    # for item in title_list:
    for item in s_title_num_char:
        pattern = re.compile(item+r"[ ]*?[^ ]+?[ ]")
        tmp=pattern.finditer(text)
        for i in tmp:
            title_list.append(i.group())
            text_start_iter_list.append(i.span(0)[0])
            title_type_list.append(2)
            text_end_iter_list.append(i.span(0)[1])

    title_list.append("引言")
    title_type_list.append(1)
    text_start_iter_list.append(0)
    text_end_iter_list.append(0)

    result_df=pd.DataFrame([title_list,title_type_list,text_start_iter_list,text_end_iter_list]).T.sort_values(by=2).reset_index(drop=True)
    # print(result_df)
    return result_df

def get_title_text(text,title_df):
    # print(title_df)
    title_1_df=title_df[title_df[1]==1]
    text_iter_list=[]
    text_list=[]
    # print(title_1_df)
    for iter1,iter2 in title_1_df[[2,3]].values:
        # print(iter1)
        if(len(text_iter_list)!=0):
            text_iter_list.append(iter1)
        text_iter_list.append(iter2)
    # text_iter_list.append(text_iter_list[len(text_iter_list)-1])
    text_iter_list.append(len(text))
    for index in range(int(len(text_iter_list)/2)):
        text_list.append(text[text_iter_list[2*index]:text_iter_list[2*index+1]])
    
    title_1_df[4]=text_list

    return title_1_df.reset_index(drop=True)

from fuzzywuzzy import fuzz
def judge_title(sample_id=0,text=r"test\n"):
    # print(text)
    text=text.replace(r"\n","")
    title_df=get_title(text)
    title_df["sample_id"]=[sample_id for x in range(title_df.shape[0])]
    # print(title_df)
    title_1_df=get_title_text(text,title_df)[["sample_id",0,1,2,3,4]]

    

    global val_df
    global train_outputs
    val_true_name=train_outputs[train_outputs["sample_id"]==sample_id]["理财产品名称"]
    
    index=0
    neg_index=[]
    for title_des in title_1_df[0].values:
        for item in title_neg_words:
            if re.search(item,title_des) is not None:
                neg_index.append(index)
                break
        index+=1


    return title_1_df.drop(neg_index)
    # print(title_list)

judge_title_result=None


for sample_id,text in tqdm(val_df[["sample_id","text"]].values):
    # print(sample_id)
    # print(text)
    judge_title_result= judge_title(sample_id,text) if judge_title_result is None else pd.concat([judge_title_result,judge_title(sample_id,text)])

# judge_title_result.to_excel("训练集段落标题分类结果.xlsx",index=None)


100%|██████████| 1800/1800 [00:19<00:00, 92.93it/s]


#fuzz裁剪，待优化

from fuzzywuzzy import fuzz
index=0
invalid_list=[]
for sample_id,first_line in tqdm(tmp_table_column[["sample_id","first_line"]].values):
    current_judge_title=judge_title_result[judge_title_result["sample_id"]==sample_id]
    judge_flag=[]
    for item in first_line:
        judge_flag=[]
        for i in current_judge_title[4].values:
            judge_flag.append(fuzz.partial_token_sort_ratio(item,i))
        if(np.mean(judge_flag)>0):
            print(np.mean(judge_flag))
            invalid_list.append(index)
            break
    index+=1
# invalid_list

#单个理财产品行划分

In [11]:

from src.time_extractor import TimeFinder
import datetime
a='2018年6月24日，2018年6月24日'
t = TimeFinder()
time_all = t.find_time(a)
print(time_all)

a="20.01万元"
re.sub("[^0-9.]","",a)
a="1"
a=a+"天"
a=["1","2","3"]
b=("").join(i for i in a)



['2018-06-24', '2018-06-24']


'20.01'

In [209]:
from fuzzywuzzy import fuzz
from src.time_extractor import TimeFinder
import datetime


#寻找居中列，即只占一行的列
def find_min_columns(df):
    count_list=[]
    result=[]
    df.copy().T.apply(lambda x:count_list.append(get_valid_columns_num(x.T)),axis=1)
    # df.copy().T.apply(lambda x:print(x),axis=1)
    # print(count_list)
    max_count=np.max(count_list)
    while (0 in count_list):
        count_list[count_list.index(0)]=max_count
    min_count=np.min(count_list)
    index=0
    for item in count_list:
        if item ==min_count:
            result.append(index)
        index+=1
    # print(result)   
    return result
def judge_time_exist(df):
    judge_flag=[]
    t = TimeFinder()
    df.map(lambda x:judge_flag.append(t.find_time(x) is not None))
    if True not in judge_flag:
        return False
    else:
        return True
def get_each_product_row(columns_list,df):
    global column_neg_words
    global window_size
    if df.shape[0]==0:
        return None
    df=df.applymap(lambda x:str(x).replace(" ",""))
    max_valid_index_num=get_valid_columns_num(columns_list)

    #字段个数不足，跳过
    if(columns_list.shape[0]<=4):
        # print(1)
        return None

    #存在敏感词，即为无效答案，跳过
    judge_flag=[]    
    for word in column_neg_words:
        columns_list.map(lambda x:judge_flag.append(fuzz.partial_ratio(word,x)==100))
    if(True in judge_flag):
        # print(2)
        return None
    sum_rows=df.head(0)
    each_row=None

    #获取居中列
    min_columns_list=find_min_columns(df)
    max_index=-1
    # print(min_columns_list)
    for index in range(df.shape[0]):
        # print(str(index)+":------")
        # print(max_index)
        if index<=max_index:
            continue
        tmp_row=df.loc[index]
        # print(tmp_row.shape)
        tmp_valid_num=get_valid_columns_num(tmp_row)
        # print(tmp_valid_num)
        # print(max_valid_index_num)
        if tmp_valid_num>=max_valid_index_num-window_size:#第一种情况，列完整
            #判断是否存在时间
            # print(tmp_row)
            if(judge_time_exist(tmp_row)):
                each_row=tmp_row
                sum_rows=pd.concat([sum_rows,each_row.to_frame().T])
                each_row=None
                continue
            else:#不存在时间则视为不完整,加入each_row之后判断是否存在时间，存在即完整
                if each_row is None:
                    each_row=tmp_row
                else:
                    each_row=each_row+tmp_row
                if(judge_time_exist(each_row)):
                    sum_rows=pd.concat([sum_rows,each_row.to_frame().T])
                    each_row=None
                    continue
                else:
                    continue
                
        elif(tmp_valid_num==1 and sum_rows.shape[0]==0):#第二种情况，列不完整,且长度为1,且sum_row无内容，认为是少数的错误字段遗留数据，舍弃
            continue
        elif((index+1)!=df.shape[0]):#第三种情况，列不完整，需要多行拼加
            #寻找目前剩下的矩阵中与当前行以外存在值的最浅的居中列
            
            start_row=index+1
            remain_df=df.iloc[start_row:,min_columns_list]
            middle_index=index+1
            middle_column=min_columns_list[0]
            # print(middle_column)
            for i in list(remain_df.index):
                valid_columns_index=get_valid_columns_index(remain_df.loc[i])
                if(len(valid_columns_index))>0:
                    middle_index=i
                    middle_column=valid_columns_index[0]
                    break
            # print(index)
            # print("middle")
            # print(middle_index)

            len_df=None

            start_row=index
            end_row=middle_index*2-index+1 if middle_index*2-index+1<=df.shape[0] else df.shape[0]
            for i in range(start_row,end_row):
                # print(df.shape)
                if(get_valid_columns_num(df.loc[i])>=max_valid_index_num-window_size):#强制中断条件
                    break
                max_index=i
                len_list=df.loc[i].map(lambda x:len(re.sub("[A-Za-z]*?","",x)))
                for j in range(len(len_list)):
                    if len_list[j]==0 and len_df is not None:
                        len_list[j]=len_df.tail(1)[j]
                len_df=pd.DataFrame(len_list).T if len_df is None else pd.concat([len_df,pd.DataFrame(len_list).T]).reset_index(drop=True)
                each_row=df.loc[i] if each_row is None else each_row+df.loc[i]
            

            start_row=max_index+1
            # print(start_row)
            # print(max_index)
            # print(len_df)

            for i in range(start_row,df.shape[0]):
                # print(df.shape)
                if(get_valid_columns_num(df.loc[i])>=max_valid_index_num-window_size):#强制中断条件
                    break
                len_list=df.loc[i].map(lambda x:len(re.sub("[A-Za-z]*?","",x)))
                # print(len_df)
                len_judge=(pd.DataFrame(len_list).T-len_df.loc[len_df.shape[0]-1]).reset_index(drop=True)
                len_judge=len_judge.T
                # print(len_judge)
                if(len_judge[len_judge[0]>0].shape[0]>0):
                    break
                max_index=i
                # print("max")
                # print(max_index)
                for j in range(len(len_list)):
                    if len_list[j]==0 and len_df is not None:
                        len_list[j]=len_df.tail(1)[j]
                len_df=pd.DataFrame(len_list).T if len_df is None else pd.concat([len_df,pd.DataFrame(len_list).T]).reset_index(drop=True)
                each_row=df.loc[i] if each_row is None else each_row+df.loc[i]
            



            if(judge_time_exist(each_row)):
                sum_rows=pd.concat([sum_rows,each_row.to_frame().T])
                each_row=None
                continue
            else:
                each_row=None
                continue
    # print(sum_rows)
    return sum_rows


column_neg_words=["实际收回","收回","赎回","实际获得","实际损益","收益情况","投资盈亏","投资收益","理财盈亏","理财收益","盈亏","收益（元","收益(元","收益(万元","收益（万元","到期收益","到期收","是否到","是否已","目前状","到期情","到息情"]
sum_product_df=[]
window_size= ##到时浮动（0-2）选择行数最多的一次
index=0
index_list=[]
for sample_id,columns_list,product_df in tqdm(tmp_table_column[["sample_id","columns","product_df"]].values):
    # product_df
    if(columns_list is None or product_df is None):
        continue
    # sample_id
    each_sum_rows=get_each_product_row(columns_list,product_df.reset_index(drop=True))
    if(each_sum_rows is not None):
        sum_product_df.append(each_sum_rows)
        index_list.append(index)
    index+=1

result_matrix=tmp_table_column.iloc[index_list,:]
result_matrix["product_df"]=sum_product_df


100%|██████████| 2162/2162 [06:42<00:00,  5.37it/s]


#### 4.抽取的是单独的数据包含
#### 起息日，到息日， 金额，认购日期，产品发行方，理财产品

In [14]:
a="12(天123"

b="123(个月123123"

c="+5（年123"

re.search("\d+?[(]*[（]*[天]+?",a).group()
re.search("\d+?[(]*[（]*[个]+[月]+?",b).group()
re.search("[^\d]+\d[(]*[（]*[年]+?",c).group()

a="2018.03.05 "
b="2018.09.03"
c=a+b
t = TimeFinder()
c=sorted(t.find_time(c),reverse=True)
a=c[0]
b=c[1]

len(pd.Series(a).shape)
d1 = datetime.datetime.strptime(a, '%Y-%m-%d')
d2 = datetime.datetime.strptime(b, '%Y-%m-%d')
d = d2 - d1

delta = datetime.timedelta(days=d.days)

datetime.datetime.strftime(d1+datetime.timedelta(days=d.days), '%Y-%m-%d')
str(d.days) + '天'


a=[1,2,3,4,5,6,6,6,6,6]
a
set(a)

'12(天'

'123(个月'

'+5（年'

1

'2018-03-05'

'-182天'

[1, 2, 3, 4, 5, 6, 6, 6, 6, 6]

{1, 2, 3, 4, 5, 6}

In [210]:
    
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
    return False

def judge_type(columns):
    type_index=[]#1:产品名,2:金额,3:发行方，4:期限
    columns=columns.map(lambda x:x.replace("（","(").replace("）",")"))
    product_name_pos_words=["产品名称","产品名册","产品名","理财产品","项目名","回购名","回购品","标的名","金融产","投资项"]#"存款种类","基金类型"#不能为空
    # product_name_neg_words=["编号","代码"]
    amt_pos_words=["存款金","认购金","投资金","投入金","受托金","理财金","金额","（元","(元","(万元","（万元","(亿元","（亿元","人民币","投资规","认购规","存款规","投入规","理财规"]
    counter_name_pos_words=["受托方","银行机","机构名","合作方名","合作银","合作机","受托人","发行主","签约方","协议方","受托机","受托银","认购银","签约银","签约机","协议机","发生主","存放银","存款银","存款机","存放机","购买银","购买机","管理人","管理银","管理机","银行名","发行机","发行主","发行人","对手方","开户银","开户行","开户机"]#可以为空
    time_length_pos_words=["期限","(天)","持有时间"]
    for words in product_name_pos_words:
        judge_flag=[]
        columns.map(lambda x:judge_flag.append(fuzz.partial_ratio(words,x)==100))
        # columns.map(lambda x:judge_flag.append(fuzz.partial_ratio(words,x)==0))
        if True in judge_flag:
            type_index.append(judge_flag.index(True))
            break
    if(len(type_index)==0):
        for words in ["种类","类型","类别"]:
            judge_flag=[]
            columns.map(lambda x:judge_flag.append(fuzz.partial_ratio(words,x)==100))
            # columns.map(lambda x:judge_flag.append(fuzz.partial_ratio(words,x)==0))
            if True  in judge_flag:
                type_index.append(judge_flag.index(True))
                break
        if(len(type_index)==0):
            type_index.append(-1)
        

    for words in amt_pos_words:
        judge_flag=[]
        columns.map(lambda x:judge_flag.append(fuzz.partial_ratio(words,x)==100))
        if True  in judge_flag:
            type_index.append(judge_flag.index(True))
            break
    if(len(type_index)==1):
        type_index.append(-1)
    
    for words in counter_name_pos_words:
        judge_flag=[]
        columns.map(lambda x:judge_flag.append(fuzz.partial_ratio(words,x)==100))
        if True  in judge_flag:
            type_index.append(judge_flag.index(True))
            break
    if(len(type_index)==2):
        type_index.append(-1)
    
    for words in time_length_pos_words:
        judge_flag=[]
        columns.map(lambda x:judge_flag.append(fuzz.partial_ratio(words,x)==100))
        if True  in judge_flag:
            type_index.append(judge_flag.index(True))
            break
    if(len(type_index)==3):
        type_index.append(-1)
    
    
    return type_index
def get_answer_matrix(result_matrix,sample_id=None):

    temp_single={}
    temp_single['认购日期'] = []
    temp_single['产品起息日'] = []
    temp_single['产品到息日'] = []
    temp_single['产品期限'] = []
    temp_single['认购金额(万元)'] = []
    temp_single['产品发行方名称'] = []
    temp_single['理财产品名称'] = []
    temp_single['sample_id'] = []

    if(sample_id is not None):
        result_matrix=result_matrix[result_matrix["sample_id"]==sample_id]
    for sample_id,columns,product_df in tqdm(result_matrix[["sample_id","columns","product_df"]].values):
        
        type_index=judge_type(columns)
        # product_df
        # columns
        # type_index
        for index in product_df.index:
            tmp_df=product_df.loc[index]
            if(len(tmp_df.shape) ==2 ):
                tmp_df=tmp_df.reset_index(drop=True).loc[0]
            product_name=""
            amt=""
            counter_name=""
            pur_dt=""
            val_dt=""
            coupon_dt=""
            time_limit=""
            #产品名
            if(type_index[0]!=-1):
                if (str(grocery.predict(tmp_df.loc[type_index[0]])) == "理财产品"):
                    product_name=tmp_df.loc[type_index[0]]
            else:
                candidate_list={}
                candidate_list["理财产品"]=[]
                candidate_list["发行方"]=[]
                candidate_list["其它"]=[]
                for each_word in tmp_df.head(1):
                    if not (is_number(each_word)):
                        candidate_list[str(grocery.predict(each_word))].append(each_word)
                if(len(candidate_list["理财产品"])!=0):
                    product_name=candidate_list["理财产品"][0]
            #金额
            if(type_index[1]!=-1):
                amt=tmp_df.loc[type_index[1]].replace("（","").replace("）","").replace("(","").replace("(","").replace("元","").replace("圆","")
                type_amt=0
                if("万" in amt or "万" in columns.loc[type_index[1]]):
                    type_amt=1
                if("亿" in amt or "亿" in columns.loc[type_index[1]]):
                    type_amt=2
                amt=re.sub("[^0-9.]","",amt)
                if(is_number(amt)):
                    amt=float(amt)
                    if(type_amt==0 and amt/10000 >float(50)):
                        amt/=10000
                    if(type_amt==2):
                        amt*=10000
                # print(amt)
            else:
                candidate_list=[]
                value_list=list(tmp_df)
                for item in value_list:
                    tmp=str(item).replace("（","").replace("）","").replace("(","").replace("(","").replace("元","").replace("圆","").replace("亿","").replace("万","")
                    if(is_number(tmp)):
                        candidate_list.append(float(tmp))

                if len(candidate_list)>0:
                    real_tmp=sorted(candidate_list,reverse=True)[0]

                    for item in value_list:
                        tmp=str(item).replace("（","").replace("）","").replace("(","").replace("(","").replace("元","").replace("圆","").replace("亿","").replace("万","")
                        if(is_number(tmp) and float(tmp)==real_tmp):
                            amt=item
                            type_amt=0
                        else:
                            continue
                        if("万" in amt ):
                            type_amt=1
                        if("亿" in amt ):
                            type_amt=2
                        amt=re.sub("[^0-9.]","",amt)
                        if(is_number(amt)):
                            amt=float(amt)
                            # print(amt)
                            if(type_amt==0 and amt/10000 >float(50)):
                                amt/=10000
                            if(type_amt==2):
                                amt*=10000
                        if(amt !="" or amt!=np.nan):
                            break
            #发行方
            if(type_index[2]!=-1):
                if (str(grocery.predict(tmp_df.loc[type_index[2]])) == "发行方"):
                    counter_name=tmp_df.loc[type_index[2]]
            else:
                candidate_list={}
                candidate_list["理财产品"]=[]
                candidate_list["发行方"]=[]
                candidate_list["其它"]=[]
                for each_word in tmp_df.head(1):
                    if not (is_number(each_word)):
                        candidate_list[str(grocery.predict(each_word))].append(each_word)
                if(len(candidate_list["发行方"])!=0):
                    counter_name=candidate_list["发行方"][0]
            
            #期限
            if(type_index[3]!=-1):
                text=str(tmp_df.loc[type_index[3]])
                # print(text)
                a=re.search("\d+?[天]+?",text)
                if (a is None):
                    a=re.search("\d+?[个]+[月]+?",text)
                if a is None:
                    a=re.search("[^\d]\d[年]+?",text)
                    if(a is not None):
                        a=re.search("\d[年]+?",a.group())
                if a is None:
                    a=re.search("^\d[年]+?",text)
                if a is not None:
                    time_limit=a.group().replace("（","").replace("）","").replace("(","").replace("(","")
                else:
                    if(is_number(text)):
                        time_limit=str(text)+"天"
                        # print(time_limit)
                    else:
                        time_limit=""

            #三个日期
            # tmp_df
            value_list=[]
            noshow=tmp_df.map(lambda x:value_list.append(str(x)))
            sum_value=(" and ").join(i for i in value_list)
            t = TimeFinder()
            time_all = t.find_time(sum_value)
            # print(time_all)
            if(time_all is not None):
                time_all=sorted(list(set(time_all)),reverse=True)
                # print(len(time_all))
                # print(sum_value)
                # print(product_name)
                if(len(time_all)==1):
                    pur_dt=time_all[0]
                    val_dt=pur_dt
                elif(len(time_all)==2):
                    # time_all=sorted(list(set(time_all)),reverse=True)
                    if(re.search("随时",sum_value) is not None or re.search("工作日",sum_value)):
                        time_limit=""
                        pur_dt=time_all[1]
                        val_dt=time_all[0]
                        coupon_dt = ""
                    else:
                        pur_dt=time_all[1]
                        val_dt=time_all[1]
                        coupon_dt = time_all[0]
                        try:
                            # 相减
                            if(type_index[3]==-1 or time_limit==""):
                                d1 = datetime.datetime.strptime(val_dt, '%Y-%m-%d')
                                d2 = datetime.datetime.strptime(coupon_dt, '%Y-%m-%d')
                                d = d2 - d1
                                time_limit = str(d.days) + '天'
                        except Exception:
                            coupon_dt = ""
                            time_limit = ""
                elif(len(time_all)==3):
                        # print(time_all)
                        pur_dt=time_all[2]
                        val_dt=time_all[1]
                        coupon_dt = time_all[0]
                        try:
                            # 相减
                            d1 = datetime.datetime.strptime(pur_dt, '%Y-%m-%d')
                            d2 = datetime.datetime.strptime(val_dt, '%Y-%m-%d')
                            d = d2 - d1
                            if str(d.days)=="1":
                                d1 = datetime.datetime.strptime(val_dt, '%Y-%m-%d')
                                d2 = datetime.datetime.strptime(coupon_dt, '%Y-%m-%d')
                                d = d2 - d1
                                time_limit=str(d.days)+"天"
                            else:
                                pur_dt=time_all[2]
                                val_dt=pur_dt
                                coupon_dt=""
                        except Exception:
                            coupon_dt = ""
                            time_limit = ""
                elif(len(time_all)>4):
                        time_all=sorted(time_all)
                        pur_dt=time_all[0]
                        val_dt=""
                        coupon_dt=""
                if pur_dt!="" and coupon_dt=="" and time_limit!="":
                    try:
                        coupon_dt=datetime.datetime.strftime(datetime.datetime.strptime(pur_dt, '%Y-%m-%d')+datetime.timedelta(days=int(re.search("\d*",time_limit).group())), '%Y-%m-%d')
                    except:
                        pass

            temp_single['认购日期'].append(pur_dt)
            temp_single['产品起息日'].append(val_dt)
            temp_single['产品到息日'].append(coupon_dt)
            temp_single['产品期限'] .append(time_limit)
            temp_single['认购金额(万元)'].append(amt)
            temp_single['产品发行方名称'] .append(counter_name)
            temp_single['理财产品名称'] .append(product_name)
            temp_single['sample_id'].append(sample_id)
        
    temp_single=pd.DataFrame(temp_single)
    return temp_single

temp_single=get_answer_matrix(result_matrix)
temp_single

100%|██████████| 1122/1122 [05:03<00:00,  3.70it/s]


,认购日期,产品起息日,产品到息日,产品期限,认购金额(万元),产品发行方名称,理财产品名称,sample_id
0,2017-08-28,2017-08-28,2017-11-27,91天,10000,,,1806
1,2016-12-23,2016-12-23,2017-12-23,365天,861.85,兴业银行股份有限公司,,446
2,2016-12-23,2016-12-23,2017-12-23,365天,861.85,兴业银行股份有限公司,,446
3,2016-12-23,2016-12-23,2017-12-23,365天,861.85,兴业银行股份有限公司,,446
4,2016-12-23,2016-12-23,2017-12-23,365天,861.85,兴业银行股份有限公司,,446
...,...,...,...,...,...,...,...,...
6769,2017-03-02,,,,5.0001e+28,,,3359
6770,2017-03-02,,,,5.0001e+28,,,3359
6771,2017-03-02,,,,5.0001e+28,,,3359
6772,2017-03-02,,,,5.0001e+28,,,3359


In [178]:
sample_id=6343
# sample_id=7123
tabel=val_df[val_df["sample_id"]==sample_id]["tabel"].iloc[0]
table_result,start_rows_list,first_line_list,product_df_list=row_combine(sample_id,tabel)
index=2
table_result[index]
product_df_list_ele=product_df_list[index].reset_index(drop=True)
product_df_list_ele
# product_df_list_ele
each_sum_rows=get_each_product_row(table_result[index],product_df_list_ele.reset_index(drop=True))

each_sum_rows2=result_matrix[result_matrix["sample_id"]==sample_id]["product_df"].iloc[0]
# # each_sum_rows
each_sum_rows

0            序号
1           受托方
2          产品名称
3    委托理财金额（万元）
4          起始日期
5          终止日期
6          产品类型
7       预期年化收益率
Name: 0, dtype: object
   0               1                         2      3           4           5  \
0  4  中信银行股份有限公司无锡分行          中信理财之共赢稳健周期91天尊享  5,000  2017-11-07  2018-02-04   
1  5  中信银行股份有限公司无锡分行            中信理财之共赢稳健周期35天  5,000  2017-11-10  2017-12-14   
2  6  中信银行股份有限公司无锡分行  中信理财之共赢稳健天天快车 A 款人民币理财产品  1,800  2017-11-24          ——   

          6      7  
0  非保本浮动收益型  5.00%  
1  非保本浮动收益型   4.5%  
2  非保本浮动收益型   3.5%  
0            序号
1           受托方
2          产品名称
3    委托理财金额（万元）
4          起始日期
5          终止日期
6          产品类型
7         年化收益率
8            备注
Name: 0, dtype: object
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Index: []
0            序号
1           受托方
2          产品名称
3    委托理财金额（万元）
4          起始日期
5          终止日期
6          产品类型
7       预期年化收益率
Name: 0, dtype: object
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7]
Index: []

'sample_id'

,0,1,2,3,4,5,6,7
0,4,中信银行股份有限公司无锡分行,中信理财之共赢稳健周期91天尊享,"5,000",2017-11-07,2018-02-04,非保本浮动收益型,5.00%
1,5,中信银行股份有限公司无锡分行,中信理财之共赢稳健周期35天,"5,000",2017-11-10,2017-12-14,非保本浮动收益型,4.5%
2,6,中信银行股份有限公司无锡分行,中信理财之共赢稳健天天快车 A 款人民币理财产品,"1,800",2017-11-24,——,非保本浮动收益型,3.5%


AttributeError: 'str' object has no attribute 'T'

In [24]:
temp_single.to_excel("result.xlsx",index=None)

judge_title(1067,train_df[train_df["sample_id"]==1067]["text"].iloc[0])

# 直接提取时间
# 如果出现两个时间第一个就是起息日，第二个就是到期日
# 如果出现一个时间就是起息日
# 出现的第一个money就是最后的金额
# 从这里面抽取所有序列
# 这里认为有逗号出现的就是money

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
    return False

from src.time_extractor import TimeFinder
import datetime
def get_list_data(df):
    df = list(df)
    new_df = []
    for i in tqdm(df):
        temp_df = []
        for h in i:
            new_h = []
            for digital in h:
                if ',' in digital:
                    # 这里也是为了统一数据有些是用元，有些是用万元
                    try:
                        ttt = float(digital.replace(',', '').replace('万元', '').replace('人民币', '').replace('元', ''))
                    except Exception:
                        continue
                    if ttt > 20000:
                        ttt = ttt/10000
                    new_h.append(ttt)
                else:
                    continue
            if len(new_h) == 0:
                continue
            temp_single = {}
            a = '_'.join(h)
            # 抽取时间和money
            t = TimeFinder()
            time_all = t.find_time(a)
            if time_all == None:
                continue
            rgrq = time_all[0]
            cpqxr = time_all[0]
            if len(time_all) > 1:
                try:
                    cpdxr = time_all[1]
                    # 相减
                    d1 = datetime.datetime.strptime(cpqxr, '%Y-%m-%d')
                    d2 = datetime.datetime.strptime(cpdxr, '%Y-%m-%d')
                    d = d2 - d1
                    cpqx = str(d.days) + '天'
                except Exception:
                    cpdxr = np.nan
                    cpqx = np.nan
            else:
                cpdxr = np.nan
                cpqx = np.nan
                
            # 筛选出除开数字与包含时间的列
            # 末尾是
            last_two = ['公司', '银行', '信托', '证券',  '分行', '支行', '中心', '业部', '商行', '建行']
            mowei = np.nan
            selected_bank_and_works = []
            for l in h:
                new_l = list(str(l))
                new_l_test = ''.join(l[-2:])
                if new_l_test in last_two:
                    mowei = l
                    continue
                if '资金' in l:
                    continue
                if '收益' in l:
                    continue
                if '到期' in l:
                    continue
                if ',' in l:
                    continue
                if '.' in l:
                    continue
                if '/' in l:
                    continue
                if '年' in l:
                    continue
                if '-' in l:
                    continue
                if len(l) < 4:
                    continue
                if is_number(l):
                    continue
                selected_bank_and_works.append(l)
            if len(selected_bank_and_works) < 1:
                continue
            
            temp_single['认购日期'] = rgrq
            temp_single['产品起息日'] = cpqxr
            temp_single['产品到期日'] = cpdxr
            temp_single['产品期限'] = cpqx
            temp_single['认购金额(万元)'] = new_h[0]
            temp_single['产品发行方名称'] = mowei
            temp_single['理财产品名称'] = selected_bank_and_works[0]
            temp_df.append(temp_single)
        new_df.append(temp_df)
    return new_df

val_contain_date = get_list_data(val_df_tabel)
# test_contain_data = get_list_data(test_df_tabel) 

#### 5.汇总整理数据

In [18]:
# # 将前面提取到的数据整理成对应格式
# sample_id_list = []
# rgrq_list = []
# lccp_list = []
# cpfxf_list = []
# rgje_list = []
# cpqxr_list = []
# cpdxr_list = []
# cpqx_list = []
# sjgmgsmc_list = []
# ggrq_list = []

# sample_id = list(val_df['sample_id'])
# gg = list(val_gm)
# time = list(val_time)
# for i, value in enumerate(sample_id):
#     for j in val_contain_date[i]:
#         sample_id_list.append(sample_id[i])
#         rgrq_list.append(j['认购日期'])
#         lccp_list.append(j['理财产品名称'])
#         cpfxf_list.append(j['产品发行方名称'])
#         rgje_list.append(j['认购金额(万元)'])
#         cpqxr_list.append(j['产品起息日'])
#         cpdxr_list.append(j['产品到期日'])
#         cpqx_list.append(j['产品期限'])
#         sjgmgsmc_list.append(gg[i])
#         ggrq_list.append(time[i])

# result = pd.DataFrame()
# result['sample_id'] = sample_id_list
# result['认购日期'] = rgrq_list
# result['理财产品名称'] = lccp_list
# result['产品发行方名称'] = cpfxf_list
# result['认购金额(万元)'] = rgje_list
# result['产品起息日'] = cpqxr_list
# result['产品到期日'] = cpdxr_list
# result['产品期限'] = cpqx_list
# result['实际购买公司名称'] = sjgmgsmc_list
# result['公告日期'] = ggrq_list
# val_result = result

sample_id_list = []
rgrq_list = []
lccp_list = []
cpfxf_list = []
rgje_list = []
cpqxr_list = []
cpdxr_list = []
cpqx_list = []
sjgmgsmc_list = []
ggrq_list = []

sample_id = list(test_df['sample_id'])
gg = list(test_gm)
time = list(test_time)
for i, value in enumerate(sample_id):
    for j in test_contain_data[i]:
        sample_id_list.append(sample_id[i])
        rgrq_list.append(j['认购日期'])
        lccp_list.append(j['理财产品名称'])
        cpfxf_list.append(j['产品发行方名称'])
        rgje_list.append(j['认购金额(万元)'])
        cpqxr_list.append(j['产品起息日'])
        cpdxr_list.append(j['产品到期日'])
        cpqx_list.append(j['产品期限'])
        sjgmgsmc_list.append(gg[i])
        ggrq_list.append(time[i])

result = pd.DataFrame()
result['sample_id'] = sample_id_list
result['认购日期'] = rgrq_list
result['理财产品名称'] = lccp_list
result['产品发行方名称'] = cpfxf_list
result['认购金额(万元)'] = rgje_list
result['产品起息日'] = cpqxr_list
result['产品到期日'] = cpdxr_list
result['产品期限'] = cpqx_list
result['实际购买公司名称'] = sjgmgsmc_list
result['公告日期'] = ggrq_list
test_result = result
test_result

In [199]:
def drop_judge(result,score_limit=31):
    global judge_title_result
    drop_list=[]
    index=0
    for sample_id,product_name in tqdm(result[["sample_id","理财产品名称"]].values):
        # print(sample_id)
        score_list=[]
        for text in judge_title_result[judge_title_result["sample_id"]==int(sample_id)][4].values:
            # print(text)
            score_list.append(fuzz.partial_token_sort_ratio(product_name,text))
        # print(score_list)
        if  len(score_list)>0 and np.max(pd.DataFrame(score_list)[0])<=score_limit:
            drop_list.append(index)
        index+=1 
    
    return result.copy().reset_index(drop=True).drop(drop_list)

# drop_judge(r)

In [ ]:
r

In [211]:
  def get_F1(val_pred, val_true):
      val_pred = list(val_pred)
      val_true = list(val_true)
      curr = list(set(val_pred).intersection(set(val_true)))
      R = len(curr)/len(val_true)
      P = len(curr)/len(val_pred)
      return 2*P*R/(P+R)

  r = pd.merge(val_df[['sample_id']], train_outputs, on='sample_id', how='left')
  val_true = r['sample_id'].astype(str) + r['理财产品名称'].astype(str) + r['认购金额(万元)'].astype(str) +r['产品发行方名称'].astype(str) + r['认购日期'].astype(str) + r['产品起息日'].astype(str)+ r['产品到息日'].astype(str) + r['产品期限'].astype(str)
  t_t_r=r
  r = temp_single
  noshow=r.fillna("").reset_index(drop=True)
  i=0
  i_list=[]
  for index in r.index:
      if r.loc[index].dropna().shape[0]<=5 or type(r.loc[index]["理财产品名称"]) is float or len(r.loc[index]["理财产品名称"])<2:
        i_list.append(i)
      i+=1
  r=r.drop(i_list)
  r=r.fillna("").applymap(lambda x:str(x).replace(" ",""))
  r.to_excel("result_r.xlsx",index=None)
  for i in range(30,35):
    r=drop_judge(r,i)
    t_p_r=r
    # r.to_excel("result_after_drop.xlsx",index=None)
    val_pred = r['sample_id'].astype(str) + r['理财产品名称'].astype(str) + r['认购金额(万元)'].astype(str) +r['产品发行方名称'].astype(str) + r['认购日期'].astype(str) + r['产品起息日'].astype(str)+ r['产品到息日'].astype(str) + r['产品期限'].astype(str)
    score = get_F1(val_pred, val_true)
    score
    i
  # val_pred = r['sample_id'].astype(str) + r['理财产品名称'].astype(str) + r['认购金额(万元)'].astype(str) +r['产品发行方名称'].astype(str) + r['认购日期'].astype(str) + r['产品起息日'].astype(str)+ r['产品到息日'].astype(str) + r['产品期限'].astype(str)
  # curr = list(set(list(val_pred)).difference(list(val_true)))

  # score = get_F1(val_pred, val_true)
  # score

100%|██████████| 6044/6044 [00:20<00:00, 290.67it/s]


0.09240374609781478

30

100%|██████████| 3947/3947 [00:13<00:00, 288.17it/s]


0.09296482412060302

...

0.09289676928097465

33

100%|██████████| 3530/3530 [00:14<00:00, 239.62it/s]


0.09279024177739055

34

In [206]:
t_p_r=t_p_r.fillna("null")
t_t_r=t_t_r.fillna("null")
t_t_r=t_t_r.astype(str)
# columns=["sample_id","理财产品名称","产品发行方名称","认购金额(万元)","认购日期_x","认购日期_y","产品起息日_x","产品起息日_y","产品到息日_x","产品到息日_y","产品期限_x","产品期限_y"]
columns=["sample_id","理财产品名称","产品发行方名称","认购金额(万元)","产品期限"]

t_p_r["sample_id"]=t_p_r["sample_id"].astype(str)
t_t_r["sample_id"]=t_t_r["sample_id"].astype(str)
p_vs_t=pd.merge(t_p_r,t_t_r,on=["sample_id","理财产品名称","产品发行方名称","认购金额(万元)","产品期限"])[columns]
p_vs_t.to_excel("result_p_vs_t.xlsx",index=None)

In [ ]:
r

  def get_F1(val_pred, val_true):
      val_pred = list(val_pred)
      val_true = list(val_true)
      curr = list(set(val_pred).intersection(set(val_true)))
      R = len(curr)/len(val_true)
      P = len(curr)/len(val_pred)
      return 2*P*R/(P+R)

  r = pd.merge(val_df[['sample_id']], train_outputs, on='sample_id', how='left')
  val_true = r['sample_id'].astype(str) + r['认购日期'].astype(str) + r['理财产品名称'].astype(str) + r['认购金额(万元)'].astype(str) + r['产品起息日'].astype(str)+ r['产品到息日'].astype(str) + r['产品期限'].astype(str) +r['产品发行方名称'].astype(str)

  r = drop_judge(result)
  print(r.shape)
  val_pred = r['sample_id'].astype(str) + r['认购日期'].astype(str) + r['理财产品名称'].astype(str) + r['认购金额(万元)'].astype(str) + r['产品起息日'].astype(str)+ r['产品到期日'].astype(str) + r['产品期限'].astype(str) +r['产品发行方名称'].astype(str)

  score = get_F1(val_pred, val_true)
  score

val_result_file=val_result.sort_values(by='sample_id').reset_index(drop=True)
val_true_file=pd.merge(train_outputs, val_df['sample_id'], on='sample_id',how="right").sort_values(by="sample_id").reset_index(drop=True)
# train_outputs
# val_result_file
# val_true_file
val_id_list=val_result_file["sample_id"].unique()
val_id_count={}
val_id_count["sample_id"]=[]
val_id_count["预测"]=[]
val_id_count["实际"]=[]
for item in val_id_list:
    val_id_count["sample_id"].append(item)
    val_id_count["预测"].append(val_result_file[val_result_file["sample_id"]==item].shape[0])
    val_id_count["实际"].append(val_true_file[val_true_file["sample_id"]==item].shape[0])

val_id_count=pd.DataFrame(val_id_count)
val_id_count["差值"]=val_id_count["预测"]-val_id_count["实际"]
print(val_id_count[val_id_count["差值"]==0].shape[0]/val_id_count.shape[0])
val_id_count.to_excel("验证集row数量对比.xlsx")

1.采用LSTM网络用提取好的部分跟pdf中的text做交互预测
理财类型、资金来源、实际购买公司和上市公司关系、买卖方是否有关联关系

把result_matrix的表格全变成文本


In [ ]:
train_table_dict={}
for sample_id,product_df in tqdm(result_matrix[["sample_id","product_df"]].values):
    if sample_id not in train_table_dict:
        train_table_dict[sample_id]=[]
    a= product_df.applymap(lambda x:train_table_dict[sample_id].append(str(x)))

# train_table_df={}
# train_table_df["sample_id"]=[]
# train_table_df["text"]=[]
# for item in train_table_dict:
#     tmp_text=("").join(a for a in train_table_dict[item])
#     if(len(tmp_text)!=0):
#         train_table_df["sample_id"].append(item)
#         train_table_df["text"].append(tmp_text)

# train_table_df=pd.DataFrame(train_table_df)
# "123" in train_table_df

from tgrocery import Grocery

# text_list=[]
# count=0
# sample_id_count=0
# for sample_id,product_name in tqdm(train_outputs[["sample_id","理财产品名称"]].values):
#     if sample_id in train_table_dict:
#         sample_id_count+=1
#         if(product_name in train_table_dict[sample_id]):
#             count+=1
# count
# sample_id_count

In [ ]:
print(grocery.predict("中信理财之共赢利率结构20062期人民币结构性理财产品"))

print(grocery.predict("利多多对公结构性存款固定持有期产品（1101168902）公司固定持有期JG902期"))

print(grocery.predict("中国光大银行中国光大银行上海浦东发展银行股份有限公司衡水分行本次自有资金委托理财金额总计"))

print(str(grocery.predict("2019-12-31江苏沙钢集团淮钢特钢股份有限公司")))

train_lstm_input = pd.merge(val_df, train_outputs, on='sample_id', how='left')
noshow=train_lstm_input.fillna("否")
# train_data = pd.DataFrame()
# train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) 
# train_data['label_1'] = "理财产品"

train_data = pd.DataFrame()
train_data['text_1'] = train_lstm_input[train_lstm_input["产品发行方名称"]!="否"].reset_index(drop=True)["理财产品名称"].astype(str)
train_data['label_1'] = "理财产品"

train_src=[]
for text,label in train_data[["text_1","label_1"]].values:
    train_src.append([label,text])

str(grocery.test(train_src))

In [ ]:
# "123" in train_table_df

In [ ]:
import jieba.analyse

text_list=[]
for product_name in tqdm(train_outputs["产品发行方名称"].values):
    text_list.append(str(product_name))

# text_list

a_list=[]

for x in jieba.analyse.extract_tags((",").join(i for i in text_list)):#可以再添加一个参数指定输出个数
    a_list.append(x)#直接输出关键词和词频


text_list=[]
for product_name in tqdm(train_outputs["理财产品名称"].values):
    text_list.append(str(product_name))

b_list=[]

for x in jieba.analyse.extract_tags((",").join(i for i in text_list)):#可以再添加一个参数指定输出个数
    b_list.append(x)

set(b_list).difference(a_list)

In [ ]:
# 最后一部分字段采用预测好的部分，跟提取的text做交互采用双输入lstm在dense层做交互预测最后几个字段

# train_lstm_input = pd.merge(train_df, train_outputs, on='sample_id', how='left')
result_matrix
train_lstm_input = pd.merge(train_table_df, train_outputs, on='sample_id', how='left')

train_lstm_input = train_lstm_input.fillna('否')

# label_1理财类型-10  label_2资金来源-3 label_3实际购买公司和上市公司关系-3 label_4买卖方是否有关联关系-2
from sklearn.preprocessing import LabelEncoder
label_1 = LabelEncoder()
# label_2 = LabelEncoder()
# label_3 = LabelEncoder()
# label_4 = LabelEncoder()

train_data = pd.DataFrame()
tmp=pd.DataFrame()
train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) 

# train_data['text_1'] = train_lstm_input['理财产品名称'].astype(str) + '_' + train_lstm_input['产品发行方名称'].astype(str)

train_data['text_2'] = train_lstm_input['text'].astype(str)

train_lstm_input["文本类别"]="理财产品"

train_data['label_1'] = label_1.fit_transform(train_lstm_input["文本类别"])


train_data2=train_lstm_input[train_lstm_input["产品发行方名称"]!="无"].reset_index(drop=True)

train_data2["文本类别"]="发行方"

tmp['text_1']=train_data2["产品发行方名称"].astype(str)

tmp['text_2']= train_data2["text"].astype(str)

tmp['label_1']=label_1.fit_transform(train_data2["文本类别"])

train_data = pd.concat([train_data,tmp]).reset_index(drop=True)


other_columns_list=["认购金额(万元)","认购日期"]

for item in other_columns_list:

    train_data2=train_lstm_input[train_lstm_input[item]!="无"].reset_index(drop=True)

    train_data2["文本类别"]="其他"

    tmp['text_1']=train_data2[item].astype(str)

    tmp['text_2']= train_data2["text"].astype(str)

    tmp['label_1']=label_1.fit_transform(train_data2["文本类别"])

    
    train_data = pd.concat([train_data,tmp]).reset_index(drop=True)



# train_data['label_2'] = label_2.fit_transform(train_lstm_input['资金来源'])
# train_data['label_3'] = label_3.fit_transform(train_lstm_input['实际购买公司和上市公司关系'])
# train_data['label_4'] = label_4.fit_transform(train_lstm_input['买卖方是否有关联关系'])
train_data
